In [ ]:
%matplotlib inline

In [ ]:
import math
import numpy
import utils

In [ ]:
from model_cvx import l1_cvx_mosek, l1_cvx_gurobi
from solver_mosek import l1_mosek_qp, l1_mosek_fusion_socp
from solver_gurobi import l1_gurobi_expand, l1_gurobi_nonexpand
from method_proj_grad import l1_proj_grad
from method_sub_grad import l1_sub_grad

In [ ]:
tester = utils.Tester(*utils.generate_data())
tester.set_xx(l1_cvx_mosek)

In [ ]:
utils.draw_sparse_figure(tester.u)

In [ ]:
out = tester.test(l1_cvx_mosek)
utils.draw_sparse_figure(out["value"])

In [ ]:
out = tester.test(l1_cvx_gurobi)

In [ ]:
out = tester.test(l1_mosek_qp)

In [ ]:
out = tester.test(l1_mosek_fusion_socp)

In [ ]:
out = tester.test(l1_gurobi_expand)

In [ ]:
out = tester.test(l1_gurobi_nonexpand)

In [ ]:
options = {
    "iters": 2000,
    "sep": 0,
    "mutable_learn_rate": False,
    "mutable_regularization": False,
    "backtracking": False,
    "init_lr": 0.001
}
out = tester.test(l1_proj_grad, options)
utils.draw_sparse_figure(out["value"])

In [ ]:
options = {
    "iters": 2000,
    "sep": 0,
    "mutable_learn_rate": True,
    "lr_func": utils.lr_func_frac_dimish(0.02, 10),
    "mutable_regularization": False,
    "backtracking": False,
}
out = tester.test(l1_proj_grad, options)

In [ ]:
options = {
    "iters": 2000,
    "sep": 0,
    "mutable_learn_rate": False,
    "mutable_regularization": False,
    "backtracking": True,
    "init_lr": 0.1,
    "beta": 0.5
}
out = tester.test(l1_proj_grad, options)

In [ ]:
options = {
    "iters": 2000,
    "sep": 0,
    "mutable_learn_rate": False,
    "mutable_regularization": True,
    "mu_func": utils.mu_func_expo_dimish(1000., tester.mu, 1500),
    "backtracking": False,
    "init_lr": 0.002
}
out = tester.test(l1_proj_grad, options)
utils.draw_sparse_figure(out["value"])

In [ ]:
options = {
    "iters": 2000,
    "sep": 0,
    "mutable_learn_rate": False,
    "mutable_regularization": False,
    "init_lr": 0.0002
}
out = tester.test(l1_sub_grad, options)
utils.draw_sparse_figure(out["value"])

In [ ]:
options = {
    "iters": 2000,
    "sep": 0,
    "mutable_learn_rate": True,
    "lr_func": utils.lr_func_frac_dimish(0.04, 20),
    "mutable_regularization": False,
    "backtracking": False
}
out = tester.test(l1_sub_grad, options)

In [ ]:
options = {
    "iters": 2000,
    "sep": 0,
    "mutable_learn_rate": False,
    "mutable_regularization": True,
    "mu_func": utils.mu_func_expo_dimish(1000., tester.mu, 1500),
    "backtracking": False,
    "init_lr": 0.0005
}
out = tester.test(l1_sub_grad, options)
utils.draw_sparse_figure(out["value"])